In [1]:
import pandas as pd
import numpy as np
import re
import requests

from concurrent.futures import ThreadPoolExecutor
import time

In [2]:
data=pd.read_excel('../DataScientists.xls')

In [3]:
data

,name,country,institution,dblp,expertise
0,aaron elmore,united states,university of chicago,https://dblp.org/pers/e/Elmore:Aaron_J=.html,NaN
1,abdalghani abujabal,germany,amazon alexa,https://dblp.org/pers/a/Abujabal:Abdalghani.html,NaN
2,abdul quamar,united states,ibm research almaden,https://dblp.org/pers/q/Quamar:Abdul.html,NaN
3,abdulhakim qahtan,netherlands,utrecht university,https://dblp.org/pid/121/4198.html,NaN
4,abhijnan chakraborty,germany,max planck institute for software systems,https://dblp.org/pers/c/Chakraborty:Abhijnan.html,NaN
...,...,...,...,...,...
1215,zixian huang,china,nanjing university,https://dblp.uni-trier.de/pers/hd/h/Huang:Zixian,NaN
1216,zoi kaoudi,qatar,qatar computing research institute,https://dblp.org/pers/k/Kaoudi:Zoi.html,NaN
1217,zoi kaoudi,germany,tu berlin,https://dblp.org/pers/k/Kaoudi:Zoi.html,NaN
1218,zsolt istvan,germany,tu darmstadt,https://dblp.org/pid/50/10703.html,NaN


In [4]:
url_list=[]
urls=data['dblp']
#print(len(urls))
#remove repeat in urls
urls=set(urls)
map_dict={}
#len(urls)

In [5]:
def url_type(url):
    t=url.split('/')[3].split('/')[0]
    return t

In [6]:
counter429=0
disappear=set()
counter=0
def get_final_url(ori_url):
    global counter429,counter
    global map_dict
    utype=url_type(ori_url)
    time.sleep(1)
    url=requests.get(ori_url,timeout=5)
    if url.status_code==429:
        time.sleep(int(url.headers['Retry-After'])+10)
        return get_final_url(ori_url)
    elif url.status_code==410 or url.status_code==404:
        print(url.status_code, url.url)
        disappear.add(ori_url)
        return None
    elif url.status_code!=200:
        print(url.status_code,url.url)
        return get_final_url(ori_url)
    elif url.status_code==200:
        counter429 =0
        counter+=1
        map_dict[ori_url]=url.url
        print(counter,url.url)
    else:
        pass
    return url.url

In [7]:
with ThreadPoolExecutor(max_workers=5) as ex:
    url_list=list(ex.map(get_final_url, urls))

1 https://dblp.org/pid/a/DJAbadi.html
2 https://dblp.org/pid/133/1954.html
3 https://dblp.org/pid/96/7128.html
4 https://dblp.uni-trier.de/pid/j/HVJagadish.html
5 https://dblp.uni-trier.de/pid/85/5637-1.html
6 https://dblp.org/pid/180/5761.html
7 https://dblp.uni-trier.de/pid/87/819-1.html
8 https://dblp.uni-trier.de/pid/71/1647.html
9 https://dblp.uni-trier.de/pid/s/DiveshSrivastava.html
10 https://dblp.org/pid/11/1466.html
11 https://dblp.uni-trier.de/pid/f/JCFreytag.html
12 https://dblp.uni-trier.de/pid/49/1457-2.html
13 https://dblp.org/pid/230/0570.html
14 https://dblp.org/pid/74/3608-6.html
15 https://dblp.uni-trier.de/pid/r/EARundensteiner.html
16 https://dblp.org/pid/34/5594.html
17 https://dblp.org/pid/133/3638.html
18 https://dblp.org/pid/67/2583.html
19 https://dblp.uni-trier.de/pid/c/VTChakaravarthy.html
20 https://dblp.org/pid/81/9436-1.html
21 https://dblp.org/pid/33/4822-12.html
22 https://dblp.org/pid/17/1242.html
23 https://dblp.org/pid/129/1674.html
24 https://dblp.or

ReadTimeout: HTTPSConnectionPool(host='dblp.uni-trier.de', port=443): Read timed out. (read timeout=5)

In [8]:
map_dict

{'https://dblp.org/pid/a/DJAbadi.html': 'https://dblp.org/pid/a/DJAbadi.html',
 'https://dblp.org/pers/z/Zeng:Daojian.html': 'https://dblp.org/pid/133/1954.html',
 'https://dblp.org/pers/r/Roy:Rishiraj_Saha.html': 'https://dblp.org/pid/96/7128.html',
 'https://dblp.uni-trier.de/pid/j/HVJagadish.html': 'https://dblp.uni-trier.de/pid/j/HVJagadish.html',
 'https://dblp.uni-trier.de/pid/85/5637-1.html': 'https://dblp.uni-trier.de/pid/85/5637-1.html',
 'https://dblp.org/pid/180/5761.html': 'https://dblp.org/pid/180/5761.html',
 'https://dblp.uni-trier.de/pers/hd/k/Kumar_0001:Niraj': 'https://dblp.uni-trier.de/pid/87/819-1.html',
 'https://dblp.uni-trier.de/pers/r/Roy:Sambuddha.html': 'https://dblp.uni-trier.de/pid/71/1647.html',
 'https://dblp.uni-trier.de/pid/s/DiveshSrivastava.html': 'https://dblp.uni-trier.de/pid/s/DiveshSrivastava.html',
 'https://dblp.org/pid/11/1466.html': 'https://dblp.org/pid/11/1466.html',
 'https://dblp.uni-trier.de/pers/f/Freytag:Johann=Christoph.html': 'https://

In [11]:
url11=[]
for url in urls:
    if url not in map_dict:
        url11.append(url)
url11

['https://dblp.org/pid/92/2769.html',
 'https://dblp.org/pers/h/Hui:Kai.html',
 'https://dblp.org/pers/w/Weidlich:Matthias.html',
 'https://dblp.org/pers/z/Zhang:Ruqing.html',
 'https://dblp.uni-trier.de/pers/c/Chakraborty:Anirban.html',
 'https://dblp.org/pid/284/0968.html',
 'https://dblp.org/pers/m/Mansour:Essam.html',
 'https://dblp.org/pid/39/1380.html',
 'https://dblp.uni-trier.de/pers/hd/n/Nikolic:Milos',
 'https://dblp.org/pid/148/7268.html',
 'https://dblp.uni-trier.de/pers/t/Tao:Yufei.html',
 'https://dblp.uni-trier.de/pers/p/Petrov:Ilia.html',
 'https://dblp.org/pid/161/0102.html',
 'https://dblp.org/pers/g/Georgakopoulos:Dimitrios.html',
 'https://dblp.org/pid/98/5721.html',
 'https://dblp.org/pers/j/Jung:Hyungsoo.html',
 'https://dblp.uni-trier.de/pers/b/Barbosa:Denilson.html',
 'https://dblp.dagstuhl.de/pid/y/TingYu.html',
 'https://dblp.uni-trier.de/pid/f/EdwardAFox.html',
 'https://dblp.uni-trier.de/pers/f/Fujita:Sumio.html',
 'https://dblp.uni-trier.de/pers/w/Wen:Ji=Ro

In [12]:
with ThreadPoolExecutor(max_workers=5) as ex:
    url_list=list(ex.map(get_final_url, url11))

404 https://dblp.org/pid/92/2769.html
410 https://dblp.org/pers/h/Hui:Kai.html
410 https://dblp.org/pers/w/Weidlich:Matthias.html
410 https://dblp.org/pers/z/Zhang:Ruqing.html
410 https://dblp.uni-trier.de/pers/c/Chakraborty:Anirban.html
404 https://dblp.org/pid/284/0968.html
410 https://dblp.org/pers/m/Mansour:Essam.html
410 https://dblp.org/pid/39/1380.html
410 https://dblp.uni-trier.de/pers/hd/n/Nikolic:Milos
410 https://dblp.org/pid/148/7268.html
410 https://dblp.org/pers/g/Georgakopoulos:Dimitrios.html
410 https://dblp.org/pid/161/0102.html
410 https://dblp.org/pid/98/5721.html
410 https://dblp.uni-trier.de/pers/t/Tao:Yufei.html
410 https://dblp.uni-trier.de/pers/p/Petrov:Ilia.html
410 https://dblp.org/pers/j/Jung:Hyungsoo.html
410 https://dblp.uni-trier.de/pers/b/Barbosa:Denilson.html
410 https://dblp.dagstuhl.de/pid/y/TingYu.html
1041 https://dblp.uni-trier.de/pid/f/EdwardAFox.html
1042 https://dblp.uni-trier.de/pid/46/1330.html
1043 https://dblp.org/pid/127/3025.html
1044 https

In [14]:
import pickle
with open('../Result/map_url.pkl','wb') as f:
    pickle.dump(map_dict,f)

In [10]:
print(len(url_list))
url_list=set(url_list)
print(len(url_list))
url_list=list(url_list)
urls1=pd.Series(url_list)


1079
1059


In [25]:
#remove None
urls1=urls1.dropna()
print(len(urls1))
urls1.to_csv('../Result/cleaned_url.csv',index=False)

1058


In [22]:
disappear=list(disappear)
disappear=pd.Series(disappear)
disappear=disappear.dropna()
disappear.to_csv('../Result/disappear.csv',index=False)

In [26]:
urls1

0       https://dblp.uni-trier.de/pid/66/3027.html
1                https://dblp.org/pid/06/9379.html
2                https://dblp.org/pid/07/1181.html
3             https://dblp.org/pid/m/RJMiller.html
4       https://dblp.uni-trier.de/pid/84/2154.html
                           ...                    
1054            https://dblp.org/pid/124/7268.html
1055            https://dblp.org/pid/139/1128.html
1056            https://dblp.org/pid/191/6085.html
1057    https://dblp.uni-trier.de/pid/32/6122.html
1058             https://dblp.org/pid/73/2015.html
Length: 1058, dtype: object

In [27]:
disappear

0                  https://dblp.org/pers/h/Hui:Kai.html
1        https://dblp.org/pers/w/Weidlich:Matthias.html
2     https://dblp.uni-trier.de/pers/p/Petrov:Ilia.html
3            https://dblp.org/pers/m/Mansour:Essam.html
4                     https://dblp.org/pid/98/5721.html
5     https://dblp.uni-trier.de/pers/b/Barbosa:Denil...
6            https://dblp.dagstuhl.de/pid/y/TingYu.html
7                    https://dblp.org/pid/148/7268.html
8                     https://dblp.org/pid/92/2769.html
9                     https://dblp.org/pid/39/1380.html
10           https://dblp.org/pers/j/Jung:Hyungsoo.html
11      https://dblp.uni-trier.de/pers/t/Tao:Yufei.html
12            https://dblp.org/pers/z/Zhang:Ruqing.html
13                   https://dblp.org/pid/284/0968.html
14                   https://dblp.org/pid/161/0102.html
15    https://dblp.uni-trier.de/pers/hd/n/Nikolic:Milos
16    https://dblp.uni-trier.de/pers/c/Chakraborty:A...
dtype: object